# bert-financial-sentiment-dis-launcher

----

In [ ]:
# 导入必要的库，包括 os 用于操作文件系统，numpy 用于处理数组，pandas 用于处理数据表格，以及 sagemaker 用于使用 Amazon SageMaker 服务。
import os 

import numpy as np 

import pandas as pd 

import sagemaker 

# 创建一个 Amazon SageMaker 会话。
sagemaker_session = sagemaker.Session() 

# 设置存储桶名称，将数据上传到该存储桶。
bucket = "<bucket name>" 

# 设置存储桶中文件的前缀，用于组织存储桶中的文件。
prefix = "sagemaker/pytorch-bert-financetext" 

# 获取执行角色，用于授权 SageMaker 访问其他 AWS 资源。
role = sagemaker.get_execution_role() 

# 将训练数据上传到 S3 存储桶中，文件路径为 './data/train.csv'。
inputs_train = sagemaker_session.upload_data("./data/train.csv", bucket=bucket, key_prefix=prefix) 

# 将测试数据上传到 S3 存储桶中，文件路径为 './data/test.csv'。
inputs_test = sagemaker_session.upload_data("./data/test.csv", bucket=bucket, key_prefix=prefix)

**解释：**

这段代码使用 Amazon SageMaker 服务，将本地训练数据和测试数据上传到指定的 S3 存储桶中，为后续的模型训练和评估做好准备。

* **导入库：** 代码首先导入了必要的 Python 库，包括：
    - `os`：用于操作文件系统，例如获取文件路径等。
    - `numpy`：用于进行数值计算，提供了数组和矩阵等数据结构。
    - `pandas`：用于数据分析和处理，提供了 DataFrame 等数据结构，方便进行数据清洗、转换等操作。
    - `sagemaker`： Amazon SageMaker 的 Python SDK，提供了用于训练、部署和管理机器学习模型的 API。
* **创建 SageMaker 会话：**  使用 `sagemaker.Session()` 创建一个 SageMaker 会话对象 `sagemaker_session`。 SageMaker 会话对象用于管理 SageMaker 资源和操作，例如创建训练任务、部署模型等。
* **配置存储桶信息：** 设置存储桶名称 `bucket` 和文件前缀 `prefix`。`bucket` 是指存储数据的 S3 存储桶的名称，需要替换为实际使用的存储桶名称；`prefix` 是指存储在 S3 存储桶中的文件路径前缀，用于组织和管理存储桶中的文件。
* **获取 IAM 角色：** 使用 `sagemaker.get_execution_role()` 获取 SageMaker 执行角色 `role`。 IAM 角色用于授予 SageMaker 访问其他 AWS 资源的权限，例如访问 S3 存储桶、读取训练数据等。
* **上传数据：**  使用 `sagemaker_session.upload_data()` 方法将本地训练数据文件 `./data/train.csv` 和测试数据文件 `./data/test.csv` 上传到指定的 S3 存储桶中。`upload_data()` 方法会返回上传数据的 S3 路径，存储在 `inputs_train` 和 `inputs_test` 变量中。

这段代码的作用是将本地数据上传到 Amazon S3 存储桶中，以便在后续使用 SageMaker 进行模型训练和评估时，可以直接从 S3 存储桶中读取数据，而无需将数据复制到 SageMaker 实例中。


----

In [ ]:
# 导入 sagemaker.pytorch 中的 PyTorch 类，用于创建 PyTorch 估计器。
from sagemaker.pytorch import PyTorch 

# 设置模型训练输出路径，训练好的模型将被保存到该路径。
output_path = f"s3://{bucket}/{prefix}" 

# 创建一个 PyTorch 估计器，用于在 SageMaker 上训练 PyTorch 模型。
estimator = PyTorch( 

    # 设置模型训练脚本的入口点。
    entry_point="train-dis.py", 

    # 设置包含训练脚本的本地目录。
    source_dir="code", 

    # 设置执行角色，用于授予 SageMaker 访问其他 AWS 资源的权限。
    role=role, 

    # 设置 PyTorch 框架版本。
    framework_version="1.6", 

    # 设置 Python 版本。
    py_version="py3", 

    # 设置用于训练的实例数量。
    instance_count=2,  

    # 设置用于训练的实例类型。
    instance_type= "ml.g4dn.12xlarge", 

    # 设置模型训练输出路径。
    output_path=output_path, 

    # 设置超参数，用于控制模型训练过程。
    hyperparameters={ 

        # 设置训练的 epochs 数量。
        "epochs": 10, 

        # 设置学习率。
        "lr" : 5e-5, 

        # 设置类别数量。
        "num_labels": 3, 

        # 设置训练数据文件名。
        "train_file": "train.csv", 

        # 设置测试数据文件名。
        "test_file" : "test.csv", 

        # 设置最大序列长度。
        "MAX_LEN" : 315, 

        # 设置训练批次大小。
        "batch_size" : 64, 

        # 设置测试批次大小。
        "test_batch_size" : 10, 
        
        # 设置后端使用 nccl 进行多机分布式训练
        "backend": "nccl"
    },
)

# 使用指定的训练和测试数据启动模型训练。
estimator.fit({"training": inputs_train, "testing": inputs_test})

**解释：**

这段代码使用 Amazon SageMaker 的 PyTorch estimator 在云端训练一个 PyTorch 模型，并使用 nccl 进行多机分布式训练，加速模型训练。

1. **创建 PyTorch Estimator：** 使用 `sagemaker.pytorch.PyTorch` 创建一个 PyTorch estimator，并设置模型训练的各种参数，例如：训练脚本的入口点、PyTorch 和 Python 版本、实例数量和类型、训练数据和测试数据的位置等。
2. **设置超参数：**  `hyperparameters` 参数是一个字典，用于传递给训练脚本的超参数，例如学习率、训练轮数、批次大小等。
3. **启动训练任务：**  调用 `estimator.fit()` 方法启动模型训练，并传入训练数据和测试数据的 S3 路径。

这段代码展示了如何在 Amazon SageMaker 上进行 PyTorch 模型的分布式训练，利用云计算资源加速模型训练过程。 


----

In [ ]:
# 导入 sagemaker.pytorch.model 中的 PyTorchModel 类，用于从训练好的模型创建模型对象。
from sagemaker.pytorch.model import PyTorchModel 

# 获取训练好的模型数据。
model_data = estimator.model_data 

# 使用训练好的模型数据、执行角色和其他配置信息创建一个 PyTorchModel 对象。
pytorch_model = PyTorchModel(
    # 训练好的模型数据。
    model_data=model_data, 

    # 执行角色，用于授权模型访问其他 AWS 资源。
    role=role, 

    # PyTorch 框架版本。
    framework_version="1.6", 

    # 包含模型推理代码的本地目录。
    source_dir="code", 

    # Python 版本。
    py_version="py3", 

    # 模型推理脚本的入口点。
    entry_point="inference.py"
) 

# 部署模型到 SageMaker Hosting 服务，创建一个端点用于实时推理。
predictor = pytorch_model.deploy(
    # 初始实例数量。
    initial_instance_count=1, 
    
    # 实例类型。
    instance_type="ml.m4.xlarge"
)

**解释：**

这段代码将使用训练好的 PyTorch 模型部署到 Amazon SageMaker Hosting 上，创建一个端点用于实时推理。

1. **创建 `PyTorchModel` 对象：**  使用 `sagemaker.pytorch.model.PyTorchModel`  类创建了一个 `PyTorchModel` 对象，用于表示待部署的模型。`PyTorchModel` 对象需要提供以下信息：
    * `model_data`：训练好的模型数据，存储在 S3 中的路径。
    * `role`：IAM 角色，用于授权 SageMaker 访问其他 AWS 资源，例如 S3、CloudWatch 等。
    * `framework_version`：PyTorch 框架版本。
    * `source_dir`：包含模型推理代码的本地目录，例如 `inference.py` 文件所在的目录。
    * `py_version`：Python 版本。
    * `entry_point`：模型推理脚本的入口点，例如 `inference.py` 文件。

2. **部署模型：** 调用 `pytorch_model.deploy()`  方法将模型部署到 SageMaker Hosting 上，并创建一个端点用于实时推理。`deploy()`  方法需要提供以下信息：
    * `initial_instance_count`：初始实例数量，用于处理推理请求。
    * `instance_type`：实例类型，指定用于运行模型的计算资源。


这段代码的作用是将训练好的 PyTorch 模型部署到 Amazon SageMaker，并创建一个可以实时进行推理的端点。该端点可以接收新的数据，并使用训练好的模型进行预测，并将预测结果返回给用户。


----

In [ ]:
# 设置预测器的序列化器为 JSONSerializer，用于将预测请求序列化为 JSON 格式。
predictor.serializer = sagemaker.serializers.JSONSerializer() 

# 设置预测器的反序列化器为 JSONDeserializer，用于将预测结果反序列化为 JSON 格式。
predictor.deserializer = sagemaker.deserializers.JSONDeserializer() 

# 使用部署好的模型对输入文本进行预测，并将预测结果存储在 result 变量中。
result = predictor.predict("The market is doing better than last year") 

# 获取预测结果中概率最大的类别索引，并打印预测结果。
print("预测类别: ", np.argmax(result, axis=1)) 

**解释：**

这段代码使用部署好的模型对输入文本进行预测，并打印预测结果。

* **设置序列化器和反序列化器：**
    * `predictor.serializer = sagemaker.serializers.JSONSerializer()`：设置预测器的序列化器为 `JSONSerializer`。序列化器用于将 Python 对象转换为字节流，以便通过网络传输或存储。`JSONSerializer` 会将预测请求数据序列化为 JSON 格式。
    * `predictor.deserializer = sagemaker.deserializers.JSONDeserializer()`： 设置预测器的反序列化器为 `JSONDeserializer`。反序列化器用于将字节流转换回 Python 对象。 `JSONDeserializer` 会将模型返回的预测结果（JSON 格式）反序列化为 Python 对象。 
* **进行预测：**  `result = predictor.predict("The market is doing better than last year")` 使用部署好的模型对输入文本 `"The market is doing better than last year"` 进行预测，并将预测结果存储在 `result` 变量中。
* **处理预测结果：**  `print("预测类别: ", np.argmax(result, axis=1))`  打印预测结果。`np.argmax(result, axis=1)` 用于获取预测结果中概率最大的类别索引。

这段代码演示了如何使用部署好的 SageMaker 模型进行预测，并使用 `JSONSerializer` 和 `JSONDeserializer` 处理请求和响应数据。


----

In [ ]:
# 删除已部署的 SageMaker 模型端点，释放资源。
predictor.delete_endpoint() 

**解释：**

这段代码使用 `predictor.delete_endpoint()` 方法删除之前部署的 SageMaker 模型端点。

* **`predictor.delete_endpoint()`**： 是 SageMaker Python SDK 提供的方法，用于删除已部署的模型端点。

**作用：**

删除已部署的 SageMaker 模型端点，以释放与该端点关联的计算资源，避免产生不必要的费用。当模型不再需要提供在线预测服务时，应该及时删除端点。


**代码逻辑：**

* 调用 `predictor` 对象的 `delete_endpoint()` 方法，该方法会向 SageMaker 发送请求，删除与该预测器关联的模型端点。

**注意：**

删除端点是一个不可逆的操作，一旦删除，就不能再使用该端点进行预测。在删除端点之前，请确保不再需要使用该端点。


----